<a href="https://colab.research.google.com/github/luvsurve/Movie_Data/blob/main/Movie_Database_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importing Required Libraries

In [38]:
import requests 
import json
import pandas as pd
import time

###Initialising Links

In [49]:
base_url = "https://api.themoviedb.org/3/movie/top_rated?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US&page="
genre_url = "https://api.themoviedb.org/3/genre/movie/list?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US"

###Genre ID list

In [ ]:
genre_data = requests.get(genre_url)
genre_result = genre_data.json()['genres']
id = []
genre_name = []
for genre in genre_result:
  id.append(genre['id'])
  genre_name.append(genre['name'])
genre_df = pd.DataFrame({'id':id,'Genre':genre_name})
genre_df

### Functions For Retreiving data from TMDB API

Genre id list to genre string 

In [47]:
def genre_txt(genre_list):
  new_list = []
  for indiv_list in genre_list:
    gen_str = ",".join([str(i) for i in indiv_list])
    new_list.append(gen_str)
  return new_list

##Retreiving movie data of single page

In [41]:
def scrape_page(pg_result):
  name = []
  overview = []
  gen = []  
  for movie in range(20):
    name.append(pg_result[movie]['original_title'])
    overview.append(pg_result[movie]['overview'] )
    gen.append(pg_result[movie]['genre_ids'])
  genre = genre_txt(gen)
  d = {'Title':name, 'Summary':overview,'Genre':genre}
  df_pg = pd.DataFrame(data=d)
  return df_pg

##Retreiving all page data

In [45]:
df = pd.DataFrame(columns=['Title','Summary','Genre'])
start = time.time()
for pg in range(1,472):
  pg_url = base_url + str(pg)
  r = requests.get(pg_url)
  data = r.json()
  result = data['results']
  df_pg = scrape_page(result)
  df = df.append(df_pg,ignore_index=True)

wait = time.time() - start
print(f"Process took: {wait} seconds")

Process took: 53.37739610671997 seconds


##Movie Data

In [48]:
df

,Title,Summary,Genre
0,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","18,80"
1,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"18,80"
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,"18,80"
3,दिलवाले दुल्हनिया ले जायेंगे,"Raj is a rich, carefree, happy-go-lucky second...","35,18,10749"
4,Schindler's List,The true story of how businessman Oskar Schind...,"18,36,10752"
...,...,...,...
9415,Premature,"On the most important day of his young life, a...",35
9416,Palais royal !,Eugenia is the queen of an imaginary European ...,35
9417,Agent Cody Banks 2: Destination London,"With all-new gadgets, high-flying action, exci...","35,28,12,10751"
9418,Venom,A group of teenagers fear for their lives in t...,"27,53"


In [74]:
df[df['Genre']=='']

,Title,Summary,Genre
3416,Anplagghed al cinema,"A queue at the ATM machine, a displaced family...",
8664,"Amore, bugie e calcetto",,


In [63]:
genre_df[genre_df['id'] == 18]['Genre'].values[0]

'Drama'

In [76]:
def genre_replace(gen_str):
  gen_list = gen_str.split(',')
  gen_name_str = []
  for id in gen_list:
    if id != '':
      genre = genre_df[genre_df['id'] == int(id)]['Genre'].values[0]
      gen_name_str.append(genre)
    gen_str = ",".join(gen_name_str)
  return gen_str

In [77]:
df['Genre'] = df['Genre'].apply(genre_replace)
df

,Title,Summary,Genre
0,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","Drama,Crime"
1,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"Drama,Crime"
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,"Drama,Crime"
3,दिलवाले दुल्हनिया ले जायेंगे,"Raj is a rich, carefree, happy-go-lucky second...","Comedy,Drama,Romance"
4,Schindler's List,The true story of how businessman Oskar Schind...,"Drama,History,War"
...,...,...,...
9415,Premature,"On the most important day of his young life, a...",Comedy
9416,Palais royal !,Eugenia is the queen of an imaginary European ...,Comedy
9417,Agent Cody Banks 2: Destination London,"With all-new gadgets, high-flying action, exci...","Comedy,Action,Adventure,Family"
9418,Venom,A group of teenagers fear for their lives in t...,"Horror,Thriller"


In [78]:
df[df['Genre']=='']

,Title,Summary,Genre
3416,Anplagghed al cinema,"A queue at the ATM machine, a displaced family...",
8664,"Amore, bugie e calcetto",,


In [79]:
df['Genre'][df['Title']=='Anplagghed al cinema'] = "Comedy,Fantasy"
df['Genre'][df['Title']=='Amore, bugie e calcetto'] = "Comedy"

In [83]:
df

,Title,Summary,Genre
0,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","Drama,Crime"
1,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"Drama,Crime"
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,"Drama,Crime"
3,दिलवाले दुल्हनिया ले जायेंगे,"Raj is a rich, carefree, happy-go-lucky second...","Comedy,Drama,Romance"
4,Schindler's List,The true story of how businessman Oskar Schind...,"Drama,History,War"
...,...,...,...
9415,Premature,"On the most important day of his young life, a...",Comedy
9416,Palais royal !,Eugenia is the queen of an imaginary European ...,Comedy
9417,Agent Cody Banks 2: Destination London,"With all-new gadgets, high-flying action, exci...","Comedy,Action,Adventure,Family"
9418,Venom,A group of teenagers fear for their lives in t...,"Horror,Thriller"


In [82]:
df.to_csv('movie_db.csv',index=False)